<h1>Classifying European Money Denominations: Comparing Two Models</h1>


<h2>Table of Contents</h2>


<p>In this lab you will compare the <code>ResNet18</code> and <code>Densenet121</code></p>
<ul>
    <li><a href="#dataset">Create Dataset Class and Object</a></li>
    <li><a href="#pre">Load Pre-trained Model</a></li>
    <li><a href="#analyze">Analyze Models</a></li>
</ul>

<p>Estimated Time Needed: <b>25 min</b></p>
<hr>


<h2>Preparation</h2>


Download the datasets you needed for this lab.


In [3]:
import requests
import tarfile
import os

def download_and_extract(url, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Download the tar.gz file
    response = requests.get(url)

    # Save the downloaded file
    downloaded_file_path = os.path.join(output_dir, "data.tar.gz")
    with open(downloaded_file_path, "wb") as file:
        file.write(response.content)

    # Extract the contents of the tar.gz file
    tar = tarfile.open(downloaded_file_path, "r:gz")
    tar.extractall(output_dir)
    tar.close()

    # Delete the downloaded tar.gz file
    os.remove(downloaded_file_path)
    print(f"Data from {url} downloaded and extracted successfully!")

output_dir = "/resources/data/"
urls = [
    "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0320EN/Datasets/PyTorch/test_data_pytorch.tar.gz",
]

for url in urls:
    download_and_extract(url, output_dir)


Data from https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0320EN/Datasets/PyTorch/test_data_pytorch.tar.gz downloaded and extracted successfully!


Import the PyTorch Modules needed in the lab.


In [4]:
# Import PyTorch Modules that will be used in the lab

import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas
from torchvision import transforms
import torch.nn as nn
torch.manual_seed(0)

Import Non-PyTorch Modules


In [5]:
# Import Non-PyTorch Modules that will be used in the lab

import time
from imageio import imread
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np

<hr>


<h2 id="dataset">Create Dataset Class and Object</h2>


In this section, you use the dataset class from the last section.


The denomination, file name and the class variable for the testing data are stored in the following CSV file.


In [6]:
# Url that contains CSV files and image dataset folder

test_csv_file = 'https://cocl.us/DL0320EN_TEST_CSV'
test_data_dir = '/resources/data/test_data_pytorch/'

Use the dataset class you created in the last lab.


In [7]:
# Create Dataset Class

class Dataset(Dataset):

    # Constructor
    def __init__(self, csv_file, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.data_name = pd.read_csv(csv_file)
        self.len = self.data_name.shape[0]

    # Get Length
    def __len__(self):
        return self.len

    # Getter
    def __getitem__(self, idx):
        img_name = self.data_dir + self.data_name.iloc[idx, 2]
        image = Image.open(img_name)
        y = self.data_name.iloc[idx, 3]
        if self.transform:
            image = self.transform(image)
        return image, y

Use the constructor <code>compose</code> to perform the following sequence of transformations in the order they are given, call the object <code>composed</code>


In [8]:
# Construct the composed object for transforming the image
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
trans_step = [transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize(mean, std)]


composed = transforms.Compose(trans_step)


Create a test dataset object using the CSV file stored in the variables the <code>test_csv_file</code>. The directories are stored in the variable <code>test_data_dir</code>. Set the parameter <code>transform</code> to the object <code>composed</code>.


In [9]:
# Create a test_dataset

test_dataset = Dataset(transform=composed
                       , csv_file=test_csv_file
                       , data_dir=test_data_dir)

<hr>


<h2 id="pre">Load Pre-trained Model</h2>


Load the <code>ResNet18</code> and <code>Densenet121</code> model you created from the last section


In [14]:
# Load pre-trained model
model = torch.load("/content/resnet18_pytorch.pt")
model_des = torch.load("/content/densenet121_pytorch.pt")

Print the structures of two models. You need to answer the questions in quiz based on the output here.


In [15]:
# Print model structure

print("ResNet18:\n", model)
print("Densenet121:\n", model_des)

ResNet18:
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inpla

Create a data loader object for the test data


In [16]:
# Set Data Loader object

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=10)

<hr>


<h2 id="analyze">Analyze Models</h2>


Find the error on the test data using the <code>ResNet18</code> model.


In [17]:
# Predict the data using ResNet18 model and print out accuracy

correct = 0
accuracy = 0
N = len(test_dataset)
for x_test, y_test in test_loader:
    model.eval()
    z = model(x_test)
    _, yhat = torch.max(z.data, 1)
    correct += (yhat == y_test).sum().item()
accuracy = correct / N
print("Accuracy using ResNet18: ", accuracy)

Accuracy using ResNet18:  1.0


Find the error on the test data using the <code>Densenet121</code> model


In [18]:
# Predict the data using densene model and print out accuracy

correct = 0
accuracy = 0
N = len(test_dataset)
for x_test, y_test in test_loader:
    model_des.eval()
    z = model_des(x_test)
    _, yhat = torch.max(z.data, 1)
    correct += (yhat == y_test).sum().item()
accuracy = correct / N
print("Accuracy using Densenet121: ", accuracy)

Accuracy using Densenet121:  0.9857142857142858


What model performed better on the test data


In [ ]:
# Write your answer here
# Accuracy using ResNet18:  1.0
# Accuracy using Densenet121:  0.9857142857142858